Jupyter notebook to extract features from the 3D shapes
Written by Dominik Waibel & Niklas Kiermeyer

In [ ]:
#import dependencies
import os 
import numpy as np
from skimage.io import imread, imsave
from skimage.measure import label, regionprops
from skimage import measure
import trimesh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import trimesh
import cv2
import math
from skimage.transform import resize
import numpy.linalg as linalg
from pyellipsoid import drawing
from skimage.feature import shape_index
from scipy.ndimage.measurements import center_of_mass
from skimage.measure import label, regionprops
from skimage.filters import gaussian, threshold_otsu
import copy
import seaborn as sns
import csv
from skimage.morphology import convex_hull_image
from skimage.feature import greycomatrix, greycoprops
from skimage.measure import moments, inertia_tensor_eigvals
from skimage.feature import hog
from skimage.filters import gabor
from skimage.morphology import convex_hull_image
from scipy.ndimage.morphology import binary_dilation
import mahotas
from skimage.feature import hog

In [ ]:
# set the path to the 2D segmentations ("mask") and the folder where the features should be saved
test_path = "./SHAPR_results/"
out_path = "./Blood_cell/features/"

In [ ]:
# normalize and threshold the data using Otsu's method: 
#https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html
def norm_thres(data): 
    maxd = np.max(data)
    data = data / maxd
    data = np.nan_to_num(data)
    if np.max(data) > 0:
        thresh = threshold_otsu(data)
        binary = data > thresh
    else: 
        binary = data
    return binary*1.0

In [ ]:
# calculate the mean curvature from a mesh
def mean_curvature(verts, faces, obj):
    mesh = trimesh.Trimesh(vertices = verts,faces=faces,process=False)
    com = center_of_mass(obj)
    z = int(round(np.nan_to_num(com[0])))
    x = int(round(np.nan_to_num(com[1])))
    y = int(round(np.nan_to_num(com[2])))
    com = np.array(com)
    com = np.array(com[np.newaxis,...])

    (centers, radius) = trimesh.proximity.max_tangent_sphere(mesh, com, inwards=True, normals=None, threshold=1e-06, max_iter=100)
    curve = trimesh.curvature.discrete_gaussian_curvature_measure(mesh, verts, radius)
    return np.mean(curve)

In [ ]:
# calculate the surface roughness from a mesh
def roughness(verts, faces):
    mesh = trimesh.Trimesh(vertices = verts,faces=faces,process=False)
    smesh = trimesh.smoothing.filter_humphrey(mesh)
    return np.mean(np.sqrt((np.sum((verts-smesh.vertices)**2))))

In [ ]:
# calculate the surface roughness from voxels
def roughtness2(obj): 
        smoothed = copy.deepcopy(gaussian(obj,2))
        smoothed = smoothed > 0.5
        return np.sum(np.abs(smoothed-obj))

In [ ]:
files = os.listdir(test_path)
print(len(files))

gabor_header = ["gabor"+str(i) for i in range(64)]
mesh_intertia_head = ["mesh_intertia"+str(i) for i in range(9)]
mesh_principal0_head = ["mesh_principal0"+str(i) for i in range(3)]
mesh_principal1_head = ["mesh_principal1"+str(i) for i in range(9)]

with open(out_path + 'ellipse_3dfeatures.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    #create and combine the header
    header1 = ['filename', 'volume', 'surface', "mean", 
               'xy_bounary', 'xz_boundary', 'yz_boundary', "boundary_3D",
               'shape_index', 'roughness', 'gauss_roughness', 'convexity',
               "zsum_mean", "zsum_std","zsum_max",
              "mean_faces","median_faces", "std_faces", "mean_verts","median_verts", "std_verts",
             "moment_cr0_v", "moment_cc0_v", "moment_cr1_v", "moment_cc1_v", "moment_cr2_v", "moment_cc2_v",
              "moment_cr_zsum", "moment_cc_zsum",
             "glcm_diss", "glcm_corr","inertia_eigvals0", "inertia_eigvals1", "inertia_eigvals2",
             "num_faces", "num_verts", "gabor_zsum", 
             "mesh_volume", "mesh_convex_hull_volume", "xy_convexity", "xz_convexity", "yz_convexity"]
    header = header1 + gabor_header +mesh_intertia_head+ mesh_principal0_head + mesh_principal1_head
    writer.writerow(header)
    for index, file in enumerate(files):
        print(index, file)
        #read the data
        data_in = np.squeeze(imread(test_path + file))
        if np.max(data_in) > 0:
            print("max", np.max(data_in))
            obj_auto_pred = np.nan_to_num(norm_thres(np.squeeze(data_in)))
            #calcualte the convexity for each central slice 
            xy_convexity = np.sum(convex_hull_image(obj_auto_pred[32,:,:])-obj_auto_pred[32,:,:])
            xz_convexity = np.sum(convex_hull_image(obj_auto_pred[:,32,:])-obj_auto_pred[:,32,:])
            yz_convexity = np.sum(convex_hull_image(obj_auto_pred[:,:,32])-obj_auto_pred[:,:,32])
            # calculate the boundary lenght for each central slice
            k = np.ones((3,3),dtype=int)
            obj = copy.deepcopy(obj_auto_pred)
            k = np.ones((3,3),dtype=int)
            obj = obj.astype(int)
            xy_boundary = np.sum(binary_dilation(obj[32,:,:]==0, k) & obj[32,:,:])
            yz_boundary = np.sum(binary_dilation(obj[:,:,32]==0, k) & obj[:,:,32])
            xz_boundary = np.sum(binary_dilation(obj[:,32,:]==0, k) & obj[:,32,:])
            #calculate the 3D boundary
            boundary_3D = np.sum(binary_dilation(obj==0, np.ones((3,3,3),dtype=int)) & obj)
            #calculate the pixel sum of the z-projection
            obj_auto_pred_zsum = np.sum(obj_auto_pred, axis = 0)
            #calculate features from the z-projection
            zsum_mean = np.mean(obj_auto_pred_zsum)
            zsum_std = np.std(obj_auto_pred_zsum)
            zsum_max = np.max(obj_auto_pred_zsum)
            #calculate glcm features
            glcm = greycomatrix(obj_auto_pred_zsum.astype("uint8"), distances=[5], angles=[0], levels=256,
                        symmetric=True, normed=True)
            glcm_diss = greycoprops(glcm, 'dissimilarity')[0, 0]
            glcm_corr = greycoprops(glcm, 'correlation')[0, 0]
            #calculate the moments of the z-projection
            m_zsum = measure.moments(obj_auto_pred_zsum)
            cr_zsum = m_zsum[0, 1] / m_zsum[0, 0]
            cc_zsum = m_zsum[1, 0] / m_zsum[0, 0]
            #calculate the gabor features of the z-projection
            gabor_zsum = np.sum(gabor(obj_auto_pred_zsum,1)[0])
            #calculate the volume
            auto_pred_vol = np.sum(obj_auto_pred)
            #calculate the convexity and check that not online z=32 has pixel values>0
            if(np.max(obj_auto_pred[:,:,33])>0 or np.max(obj_auto_pred[:,:,34])>0):
                convexity = np.sum(convex_hull_image(obj_auto_pred)-obj_auto_pred)
            else:
                convexity = 0
            #calculate the moments
            m_v = measure.moments(obj_auto_pred)
            cr0_v = m_v[0][0, 1] / m_v[0][0, 0]
            cc0_v = m_v[0][1, 0] / m_v[0][0, 0]
            cr1_v = m_v[1][0, 1] / m_v[1][0, 0]
            cc1_v = m_v[1][1, 0] / m_v[1][0, 0]
            cr2_v = m_v[2][0, 1] / m_v[2][0, 0]
            cc2_v = m_v[2][1, 0] / m_v[2][0, 0]
            #calculate the faces and vertices describing the 3D object
            verts_pred, faces_pred, _, _ = measure.marching_cubes_lewiner(obj_auto_pred*255.)
            #calculate the surface from the faces and vertices
            surface_pred = measure.mesh_surface_area(verts_pred, faces_pred)
            #calculate the gabor features
            gabor_feat = []
            for g in range(np.shape(obj_auto_pred)[0]): 
                gabor_feat.append(np.sum(gabor(obj_auto_pred[g,:,:],1)[0]))
            #calculate the inertia eigenvalues
            inertia_eigvals = inertia_tensor_eigvals(obj_auto_pred)
            #calculate the mesh describing the 3D object from the faces and vertices
            mesh = trimesh.Trimesh(vertices=verts_pred,
                           faces=faces_pred,
                           process=False)
            #calculate the mesh interia and principal axes
            mesh_intertia = list(mesh.moment_inertia.flatten())
            mesh_principal = list(trimesh.inertia.principal_axis(mesh.moment_inertia))

            tri_in = trimesh.inertia.principal_axis(mesh.moment_inertia)
            mesh_principal0 = list(tri_in[0].flatten())
            mesh_principal1 = list(tri_in[1].flatten())
            #combine the features and write them to the .csv file
            feat = [file, auto_pred_vol, surface_pred, np.mean(obj_auto_pred), 
                    xy_boundary, xz_boundary, yz_boundary, boundary_3D,
                    mean_curvature(verts_pred, faces_pred,obj_auto_pred), 
                    roughness(verts_pred, faces_pred), roughtness2(obj_auto_pred), convexity, 
                    zsum_mean, zsum_std, zsum_max, 
                    np.mean(faces_pred), np.median(faces_pred), np.std(faces_pred), np.mean(verts_pred),
                    np.median(verts_pred),
                    np.std(verts_pred),
                   cr0_v, cc0_v, cr1_v, cc1_v, cr2_v, cc2_v, cr_zsum, cc_zsum,
                   glcm_diss, glcm_corr, inertia_eigvals[0], inertia_eigvals[1], inertia_eigvals[2],
                   len(faces_pred), len(verts_pred), gabor_zsum, 
                   mesh.volume, mesh.convex_hull.volume, xy_convexity, xz_convexity, yz_convexity] 

            feat = feat+ gabor_feat + mesh_intertia + mesh_principal0 + mesh_principal1
            print("len features", len(feat), len(header))
            writer.writerow(feat)
        else:
            zer =  [0] * 126
            feat = [file]
            feat = feat + zer
            writer.writerow(feat)